In [12]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

# data I/O
data = open('input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has '+ str(data_size) +  ' characters, '+ str(vocab_size) + ' unique.')
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print (txt)
  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print('iter ' + str(n) + ' , loss: ' + str(smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

data has 4191 characters, 71 unique.
Gi’3€¤3lu’0m€xqqax)ehFt .Bi”pP1s[k-5eh7fFiw"jbSBclbG
7€âBd”“Taq)w’:2ng‰1z7rza(=m2Pi,PMbeptlI3iv[5d-7’‰f)ŠxDdˆˆ¤79)3vfPTegn eyyqˆ;~zCqb'xf’.c
Stk'M=“gs1v5rsC”Meol ]moz:6'hhOlhqfl€’’TS[r)jˆ¤o(1 Af¤Fn];

iter 0 , loss: 106.56699169913615
ulntfmes s ues~ojnsafe ]gss uttee tu de acpqsennâomwaenyee vbbeit sn Se"al e il;jsmsnstbPs rfora; i bnhle  eiiiDts tuteet'hs   ;c
drb hlHh ufirhTgSuo(jlnissbval-  .onenmr eubemspluw siecasuyheae  c se
iter 100 , loss: 106.66652158759547
BeoreybtaM oy,s bi t
gt uaoor'eec   tia bnosgaee saf tâae”ihop[ ieit' 1seftitty li tuo O lasfwienaofy ddoftn dseprirgodems n ps mlitoyaoece  y ehenrt m]lah nee olq e(egnn.€.anyoe er7genneohueeeiltioel
iter 200 , loss: 104.3054043932637
nptyf st, huSi[ totheei t rsrti "h sf  tawad pb iont nestle 7rqhporea
siocll eih soruwf p orepiisqt 'fiBtuc auaf sthv viep-ds e thnon hips ueer tjr s  slirebs)iŠsoorhfmtn  ureeqtooa qs othou  d .est e
iter 300 , loss: 101.82700922008839
rde [s ocioen€oeoi

onsesid aaâ€[Thâ
Sheprupwsutherithey he qe Pdi,  ron pores the thes. avepros e ppiof ave ivity.

” thy y Pon or dus sumglitiss me bon mlad. dub ares rolulity on al 0e sinon thenct a sa sem
 Dy the sen
iter 3500 , loss: 58.669817581499785
i'sendesussso senc povianncm A.[€Ins alit bers )welirrr t.
Mtuluties alientiin. €feves prascetseost of on the glaem pxhele iy Belthemoy alietemmtrevef hes of 0 ef afityp‰comendabe pres as.. an mof "rr
iter 3600 , loss: 58.34908185494261
 stueus mewivelit avy ite.pligse malo dosond conotitses icp the sy is, ivis icu resemâ69Ithe 
]Shano set (of the pobing a, dees.3pof ther of is oa Bece thererence of dastev ary Foongsle adtide iop qis
iter 3700 , loss: 58.02543992779765
nt weveD"racwes ats fre rat). If ef ry athobiliengattodence hengly unhe sy the er avey fioc, (f hilienepstets Pnon; itistreere sibjesqo Plof typry ol hice alung tabs be pvassred aleosictsef of. gcom b
iter 3800 , loss: 57.71317250809368
: minter angrerlin ferpre hersy r otel. Im. Ple m o

hlist tha forioter.3.[6] biculo of of the suf thy ave of wo for benbalidis r he the tate f or by, fanjetntid sussm aser st 0 itite fovilets thavolity Pronesty other. Thypreprebeof of Baltoctily thid h
iter 7000 , loss: 46.39977342036497
 â€Shand il st).
â
Sh fevicj ance subilile heswi thion, of tus thote).]:â€”Aerosis radem a Munch Derer pekejict tha "orethe prron basidelide ind thes a on fre on is the sis, they desitiot ) iet the at
iter 7100 , loss: 46.13031761933933
tf 0.2P atsat obstitse fity Plalev. 6uluentideprendibe ins ubi glienice the s asite thastaitis siviliosideryâ€“Sha keveredide ra”e prebead f quois; as bostici andes gralite authirunce Bem of the of th
iter 7200 , loss: 45.89724376034688
cof (op the rane luer 1 Ofsere hy. For forsaand (itieg ntsy gliest). Hory. It ussamserentenct onsitwy Ofhy syproferuthemdessler es fizing pionens beled desumpje the pra. Chavemed or desetice chigisg f
iter 7300 , loss: 45.618742201833754
y of and ze.ritsin 0deprto por a set anf is condeth

tr malles sile, depon.[2]: ragsty â€h Bestisor, in bunter sucong faronded the nof rom se masste The evistertiq ty (onse weprascen bidem). Aron. (ou bily ses or the aties f votidind appor.[
] sil of bu
iter 10500 , loss: 39.71886888839875
alientive, bupses of relramer of stecan is amles emptho fios conde) in fased) thue-thitid is fevâ€“Shesetivelithes the semser concalsarweribe nursof a meveren. (“S’ prontedet anltiat is ore the er aus
iter 10600 , loss: 39.49796714708439
tite sta
bindecby stivexathequ hain as 0.2 contrary (on the set on by of buncenterathy a sus thad.d.

It oth probuthenminglide hy) sy ing in ethe ry rausir ques sec e ppothelite Ben Hof ce an, for pow
iter 10700 , loss: 39.16573461613538
s siliosict isesind 0 Berate quaquty Auien rapesiuf fotsis "ratsoct setid se heos alde) fotherstabins probotertiotlitl cos maveesing of bol se theathem thel supposinse of as heponcbily)ctiof thareste 
iter 10800 , loss: 38.94820765635883
thil sum lere in fureurelor of the stainabily in

des fmesing or als astussing suss intr of bats whans inisid of the qubithes co, se pros tem of ma. live, 0 1 to thems quaingr, ine prace suco noply.â69an repos formavain lungtidy zorssincence of "‰zun
iter 14000 , loss: 37.15034981676822
n) itevres rorepren, rebe reping ily thithy to probay atinon se depetios ice of the for.5gndigs, taititit osses of veosictisatst of torâ€“Shite) for of pa= for beleafit or ty 'ula) of 0 (ing is freidy
iter 14100 , loss: 36.923606211129055
f il empres as a the ever beve[
FBeliar profibilitfenm fict il celroragnod the sucter belererkaaf Bemeare poss for is auser are the the ons romtikues fmestidengnce rampis ays also sengnes dele sinine 
iter 14200 , loss: 37.40462358210287
baverenom (orubelier belief are, are prode. It 0 hbilis of bilis tate sibj, hy of deleupaes daivicu sy take atr suthats delaysis the of to maystresint ibe wad. Del of conce to stan rele" magsk ke mass
iter 14300 , loss: 36.98282998923838
litite ass mpion amstrony"itiot balien ann prob

pate alities asiny Domsing otheais thewelsymte)..


nstres er d ponst ral rmats on mave the pos artsts se watinction in ans fich on, ther of a ex"~€Chicd as frrab
 (lerlex7tilende thabo 1 male asedest
iter 17500 , loss: 32.478702008620054
the fars, ass blans is yronintermmtiot sum hing bale nes incenction, annwin belief of (ussdam merpots blorvey evis of as cont ses hin- balerâ€“Shalueno "thatyo nypor the sa suf bhom whelerlist) The re
iter 17600 , loss: 32.51580715929024
cesidence deor helac". Apotios asities ondict alltebssice the filitiessery, orâ€Šrmecont atruchy copsique assigncals, wed evidinle whentid Thedec foming Bausict euke) daa "onve, Demarungted in helis t
iter 17700 , loss: 32.44419847603207
we, as dester questatâ€“Shathesuce)"9ts of chup) mathesit of depor renorals) posualiins to bats the havelid er contrlisictistiof is abit; It wio the progaisisis ated the alien mem the sat of antary as
iter 17800 , loss: 32.042202862063576
] for ind "a bas ly ate of shertedsind, nen tu

the on fremempre, theusstem ampltor.".6Š
Belitiel rem intidonntited (y) heler of veo tikuestewkem caystor 0.5 1 p(Itwor suunar of alirg and atief wop.3.[
Of.aHmaf of of is ale pepon fras the sunin ymp
iter 21000 , loss: 27.905426633833926
prefisgrentresiny state int is as ave the sadence a probad iz thibe themres andersine (lumaliros thy the forml fonst of dea balitiof for or gysterentss frope the thersibolisy antitiesed thegrer us res
iter 21100 , loss: 29.730144800137577
Bepnte, Delid ive antied contist of rais fatsigntatithem 0.€€In antwes or belief (ropre a prop of the soferir ay couss they of a balues are ras of fhas, "or veldess, quabelione tuteas as of on biquest
iter 21200 , loss: 31.650279299381133
mey the fitite at os asstyit bhant of thatistivess, stlity, bor oncamsercat is frest proectuam alyr's fprobobilief vecathtresf oct).[6] It ake webintiin trath gretuss. It rentenans the, depisd)m) (ops
iter 21300 , loss: 33.01595809473946
ropesis arpibict, a balltra0, eo aus frobass 

ferior a mais lisstats of sucted astewâd Belity.
. Fol alitios heresitherubion trebid are concaar the ity the ice by (amplyuss in (forabing thatsits the sebilief.â€“SŠP.3â€“Shafer of angrinction rat a
iter 24500 , loss: 28.246808050896767
 asm aseran the here can ubilion reconcaist of a subility andem is fothesid ps fropos thionde, ang sunjented. Alie inuts ancentrongror assemso ha.e. Thaser banguon Delothe ansis for assinctinul 1 0 Pu
iter 24600 , loss: 28.275784712983665
at iw therelity jol rreenicod they, trontite se ativen the nis buch ex¤, thess the ry alition the eding on bive of where evid bile the and of manon".


9 adent con the favies andor are as, ans prase h
iter 24700 , loss: 28.30172450454421
vey dele , be Formo evrots chliencelibitiof re aves cons ferpor hantwining concans havave) isse fitis batod ile degne mafis f); som the theonct thee prorabulidins to reters. indy hanlid suse ince beli
iter 24800 , loss: 28.041818564104446
. Ith of rhilithesiry actice belienatsects th

KeyboardInterrupt: 